## Queries - ComCam

In this notebook, we show how to query the ComCam repository\
and view the resulting images.\
Craig Lage - 17-Nov-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
from lsst.geom import Point2D, Extent2I

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTComCam")
instrument = 'LSSTComCam'

In [ ]:
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultButler("LSSTComCam")
expId = 2024120800619
exp = butler.get('postISRCCD', detector=4, exposure=expId, instrument="LSSTComCam")

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
dayObs = 20240808
instrument = "LSSTComCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTComCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


# Get the data from the headers

In [ ]:
expId = 2024120400106
mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
for key in mData.keys():
    print(key, mData[key])

In [ ]:
for expId in range(2024112300011, 2024112300017):
    mData = butler.get('raw.metadata', detector=4, exposure=expId, instrument=instrument)
    print(expId, mData['HVBIAS'], mData['BSSVBS'])

# Now get the image data

## Next, look at the raw data from one of the exposures.
### There are generally three options, raw, postISRCCD, and calexp
### Each has successively more processing

In [ ]:
expId = 2024111700488
#exp = butler.get('raw', detector=4, exposure=expId, instrument=instrument)
exp = butler.get('postISRCCD', detector=8, exposure=expId, instrument=instrument)
#exp = butler.get('calexp', detector=1, visit=expId, instrument=instrument)
%matplotlib inline        
x = plot(exp, stretch='asinh')

## Define a simple ISR
## Maybe you want to start with the raw image and do your own ISR
### This has just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=True
isrConfig.doDefect=False
isrConfig.doNanMasking=False
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result

In [ ]:
expId = 2024121000508
detector = 8
exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
biasExp = butler.get('bias', detector=detector, exposure=expId, instrument=instrument) # This is a bias image associated with the data
ptcExp = butler.get('ptc', detector=detector, exposure=expId, instrument=instrument) # This is a bias image associated with the data
isrResult = isrTask.run(exp, bias=biasExp, ptc=ptcExp) # This runs the ISR
x = plot(isrResult.exposure, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_{expId}_{detector}.png")

# Plot a small region

In [ ]:
x = 3220; y = 2000
width = 500
center = Point2D(x, y)
extent = Extent2I(width, width)
cutout = isrResult.exposure.getCutout(center, extent)
plt.imshow(cutout.image.array, origin='lower')
#x = plot(cutout, stretch='ccs', showCompass=False)

In [ ]:
x = 950; y = 2150
width = 50
center = Point2D(x, y)
extent = Extent2I(width, width)
cutout = exp.getCutout(center, extent)
plt.imshow(cutout.image.array, origin='lower')
#x = plot(cutout, stretch='ccs', showCompass=True)

In [ ]:
fig, [ax1, ax2] = plt.subplots(2, 1)
plt.suptitle(f"ComCam {expId}, Sat Star")
ax1.plot(isrResult.exposure.image.array[2015, 3215:3245], marker='x', label='Y=2015')
ax1.plot(isrResult.exposure.image.array[2025, 3215:3245], marker='x', label='Y=2025')
ax1.plot(isrResult.exposure.image.array[2030, 3215:3245], marker='x', label='Y=2030')
ax1.legend()
slice1 = np.mean(isrResult.exposure.image.array[500:1500, 3215:3245], axis=0)
level1 = 8800
gtLevel = slice1[slice1>level1]
excess = np.sum(gtLevel) - level1 * len(gtLevel)
ltLevel = slice1[slice1<level1]
deficit = np.sum(ltLevel) - level1 * len(ltLevel)

ax2.plot(slice1, marker='x', label=f'Y below(500:1500)')#: excess={excess:.0f}, deficit={deficit:.0f}')
#ax2.axhline(level1, ls='--', color='steelblue')
slice2 = np.median(isrResult.exposure.image.array[2200:3500, 3215:3245], axis=0)
level2 = 8700
gtLevel = slice2[slice2>level2]
excess = np.sum(gtLevel) - level2 * len(gtLevel)
ltLevel = slice2[slice2<level2]
deficit = np.sum(ltLevel) - level2 * len(ltLevel)

ax2.plot(slice2, marker='x', label=f'Y above(2200:3500)')#: excess={excess:.0f}, deficit={deficit:.0f}')
#ax2.axhline(level2, ls='--', color='orange')
ax2.legend(loc='lower left')
plt.ylim(370, 440)
plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_Sat_Slices_{expId}_{detector}.png")

# Now assemble all 9 CCDs and plot the result

In [ ]:
def isrCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    biasExp = imageSource.butler.get('bias', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    ptcExp = butler.get('ptc', detector=ccd.getId(), day_obs=dayObs, 
                        seq_num=seqNum, exposure=expId, instrument=instrument)
    isrResult = isrTask.run(exp, bias=biasExp, ptc=ptcExp) # This runs the ISR
    oim = isrResult.exposure.image
    return oim

def simpleCallback(im, ccd, imageSource):
    """Assemble the CCD image."""
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=False, correctGain=False)
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTComCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
import lsst.afw.display as afwDisplay
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min='zscale')
dayObs = 20241116
seqNum = 300
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, raft="R22",
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=isrCallback,
                                                       background=np.nan),
                              binSize=4, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

#plt.savefig(f"/home/c/cslage/u/ComCam/images/ComCam_{dayObs}_{seqNum}.png")

# The cell below will save the combined image as a FITS file

In [ ]:
filename = "/home/c/cslage/u/ComCam/images/FITS_2024111600300.fits"
mos.writeFits(filename)